In [1]:
import json
import pandas as pd

In [2]:
# Read CSV data
csv_data = pd.read_csv('resources/miami_info.csv')

In [ ]:
# Read GeoJSON data
with open('resources/fl_florida_zip_codes_geo.min.json', 'r') as file:
    geojson_data = json.load(file)

In [ ]:
# Create a dictionary from CSV data for easy lookup
zipcode_income_dict = dict(zip(csv_data['Zipcode'].astype(str), csv_data['Median_income']))

In [ ]:
# Update GeoJSON data
filtered_features = []
for feature in geojson_data['features']:
    zipcode = feature['properties']['ZCTA5CE10']
    if zipcode in zipcode_income_dict:
        # Update feature with Median_income information
        feature['properties']['median_income'] = zipcode_income_dict[zipcode]
        # Ensure correct polygon order
        if feature['geometry']['type'] == 'Polygon':
            feature['geometry']['coordinates'] = [list(reversed(ring)) for ring in feature['geometry']['coordinates']]
        elif feature['geometry']['type'] == 'MultiPolygon':
            feature['geometry']['coordinates'] = [[list(reversed(ring)) for ring in polygon] for polygon in feature['geometry']['coordinates']]
        filtered_features.append(feature)


In [ ]:
# Create a new GeoJSON object with filtered features
filtered_geojson_data = {
    "type": "FeatureCollection",
    "features": filtered_features
}


In [ ]:
# Save the filtered GeoJSON data to a new file
with open('resources/filtered_zipcodes.geojson', 'w') as outfile:
    json.dump(filtered_geojson_data, outfile)